In [2]:
import pandas as pd
import numpy as np
from datetime import datetime,date

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
pd.set_option('display.max_columns', None)

In [202]:
# 2020-11-12:双状况计划监控模型——末日
def source_predict_state_1(jsondata):
    '''
    :param jsondata: {"columns": ["channel_id","source_id","plan_name", "model_run_datetime","create_time","media_id",
    "game_id","platform","source_run_date_amount","create_role_num","create_role_cost","create_role_pay_num",
    "create_role_pay_cost","create_role_pay_sum","create_role_roi","create_role_pay_rate","create_role_pay_num_cum",
    "learning_type","learning_time_dt","learning_time_hr","deep_bid_type"]}
    :return:'label','budget'
    '''
    
    # 该模型只能接受计划状态plan_run_state=1且data_win=0 且 model_run_datetime 等于当日的数据，即只接受当日实时数据
    # label为1时关计划，为0时开计划，如果模型输出label结果与计划当前状况（开/关）相同，则不操作。例如，计划当前是关停状况，当模型输出为1时，不操作
    # budget输出值为该计划预算需要增加至的数值，如果budget小于等于当前计划预算，则不操作。例如budget=6000，则将该计划的预算增加至6000元
    # 加预算操作与开关操作相互独立，例如可以存在加了预算，但不开计划的情况
    # 晚上12点至第二天早上8：00不执行加预算操作，但开关指令正常执行
    # 早上7：55分，将所有在运行的计划状态plan_run_state改为2。调用模型2进行判断，是否达标。达标则状态改为1，开计划，预算由模型1计算决定。不达标，则为关停状态，且plan_run_state为2
        
    
    

    data = pd.DataFrame(data=jsondata['data'], columns=jsondata['columns'])
   
    # 数据预处理
    data = data[data['source_run_date_amount'] > 0]  # 删除消耗为0的数据
    # 去重，对当天运行进行监控情况
    # 对消耗300元以下的不作判断，对数据按平台、win进行分类处理
    data_not_ignore = data[data['source_run_date_amount'] >= 300]

    # 增加每次付费跑法的判断
    data_pre_action = data_not_ignore[data_not_ignore['deep_bid_type'] == 'BID_PER_ACTION']
    data_not_ignore = data_not_ignore[data_not_ignore['deep_bid_type'] != 'BID_PER_ACTION']
    
    data_win_0 = data_not_ignore[data_not_ignore['platform'] == 1]
    data_win_0_ios = data_not_ignore[data_not_ignore['platform'] == 2]

    data_win_0_pre = data_pre_action[data_pre_action['platform'] == 1]
    data_win_0_ios_pre = data_pre_action[data_pre_action['platform'] == 2]

    result_df = pd.DataFrame()
    
    # 1:关; 0:开； 3：加预算
    # win=0预判
    # 每次付费安卓
    if data_win_0_pre.shape[0] != 0:
        data_win_0_pre['label'] = data_win_0_pre.apply(lambda x: 1 if (x.create_role_pay_sum == 0) &
            (x.create_role_cost >= 200) else(1 if (x.create_role_pay_sum <=24)&(x.create_role_cost >= 250) else
            (1 if (x.create_role_roi<=0.018)&(x.create_role_cost >= 300) else (1 if x.create_role_cost>=400 else 0))), axis=1)
        
        result_df = result_df.append(data_win_0_pre)

    # 每次付费ISO
    if data_win_0_ios_pre.shape[0] != 0:
        data_win_0_ios_pre['label'] = data_win_0_ios_pre.apply(lambda x: 1 if (x.create_role_pay_sum == 0) &
            (x.create_role_cost >= 300) else(1 if (x.create_role_pay_sum <=24)&(x.create_role_cost >= 350) else
            (1 if (x.create_role_roi<=0.014)&(x.create_role_cost >= 400) else (1 if x.create_role_cost>=500 else 0))), axis=1)
        
        result_df = result_df.append(data_win_0_ios_pre)
    
    # 其它付费方式安卓
    if data_win_0.shape[0] != 0:
        data_win_0['label'] = data_win_0.apply(lambda x:1 if (x.create_role_pay_sum == 0) &
            (x.create_role_cost >= 80) else(1 if (x.create_role_pay_sum <=24)&(x.create_role_cost >= 100) else
            (1 if (x.create_role_roi<=0.018)&(x.create_role_cost >= 120) else (1 if x.create_role_cost>=150 else 0))), axis=1)
        
        result_df = result_df.append(data_win_0)
    
    # 其它付费方式IOS
    if data_win_0_ios.shape[0] != 0:
        data_win_0_ios['label'] = data_win_0_ios.apply(lambda x:1 if (x.create_role_pay_sum == 0) &
            (x.create_role_cost >= 180) else(1 if (x.create_role_pay_sum <24)&(x.create_role_cost >= 200) else
            (1 if (x.create_role_roi<=0.014)&(x.create_role_cost >= 250) else (1 if x.create_role_cost>=300 else 0))), axis=1)
        
        result_df = result_df.append(data_win_0_ios)
        
    result_df_an = result_df[result_df['platform'] == 1]
    result_df_ios = result_df[result_df['platform'] == 2]
    result_df = pd.DataFrame()
    
    # 安卓加预算
    data_1 = result_df_an[result_df_an['source_run_date_amount']<=5000]
    data_2 = result_df_an[(result_df_an['source_run_date_amount']>5000)&(result_df_an['source_run_date_amount']<=10000)]
    data_3 = result_df_an[(result_df_an['source_run_date_amount']>10000)&(result_df_an['source_run_date_amount']<=20000)]
    data_4 = result_df_an[(result_df_an['source_run_date_amount']>20000)&(result_df_an['source_run_date_amount']<=46000)]
    data_5 = result_df_an[(result_df_an['source_run_date_amount']>46000)&(result_df_an['source_run_date_amount']<=90000)]
    data_6 = result_df_an[(result_df_an['source_run_date_amount']>90000)&(result_df_an['source_run_date_amount']<=140000)]
    data_7 = result_df_an[(result_df_an['source_run_date_amount']>140000)&(result_df_an['source_run_date_amount']<=200000)]
    data_8 = result_df_an[(result_df_an['source_run_date_amount']>200000)&(result_df_an['source_run_date_amount']<=250000)]
    data_9 = result_df_an[(result_df_an['source_run_date_amount']>250000)&(result_df_an['source_run_date_amount']<=300000)]
    data_10 = result_df_an[result_df_an['source_run_date_amount']>300000]
    
    if data_1.shape[0] != 0:
        data_1['budget'] = data_1.apply(lambda x: 6600 if (x.create_role_pay_sum>=60)&(x.create_role_pay_num>=2) else
                                       (6600 if x.create_role_pay_sum>=128 else np.nan), axis=1)
        result_df = result_df.append(data_1)
    if data_2.shape[0] != 0:    
        data_2['budget'] = data_2.apply(lambda x: 13000 if (x.create_role_roi>=0.02)&(x.create_role_pay_num>=2) else
                                        (13000 if x.create_role_roi>=0.3 else np.nan), axis=1)
        result_df = result_df.append(data_2)
    if data_3.shape[0] != 0:
        data_3['budget'] = data_3.apply(lambda x: 23000 if (x.create_role_roi>=0.021)&(x.create_role_pay_num>=2) else
                                        np.nan, axis=1)
        result_df = result_df.append(data_3)
    if data_4.shape[0] != 0:
        data_4['budget'] = data_4.apply(lambda x: 50000 if (x.create_role_roi>=0.021)&(x.create_role_pay_cost<=7000) else
                                       (50000 if (x.create_role_roi>=0.03)&(x.create_role_pay_cost<=9000) else 
                                        np.nan), axis=1)
        result_df = result_df.append(data_4)
    if data_5.shape[0] != 0:
        data_5['budget'] = data_5.apply(lambda x: 95000 if (x.create_role_roi>=0.022)&(x.create_role_pay_cost<=7000) else
                                       (95000 if (x.create_role_roi>=0.03)&(x.create_role_pay_cost<=9000) else 
                                        np.nan), axis=1)
        result_df = result_df.append(data_5)
    if data_6.shape[0] != 0:
        data_6['budget'] = data_6.apply(lambda x: 148000 if (x.create_role_roi>=0.022)&(x.create_role_pay_cost<=6000) else
                                       (148000 if (x.create_role_roi>=0.03)&(x.create_role_pay_cost<=8000) else 
                                        np.nan), axis=1)
        result_df = result_df.append(data_6)
    if data_7.shape[0] != 0:
        data_7['budget'] = data_7.apply(lambda x: 220000 if (x.create_role_roi>=0.023)&(x.create_role_pay_cost<=6000) else
                                       (220000 if (x.create_role_roi>=0.03)&(x.create_role_pay_cost<=8000) else 
                                        np.nan), axis=1)
        result_df = result_df.append(data_7)
    if data_8.shape[0] != 0:
        data_8['budget'] = data_8.apply(lambda x: 275000 if (x.create_role_roi>=0.024)&(x.create_role_pay_cost<=5000) else
                                       (275000 if (x.create_role_roi>=0.03)&(x.create_role_pay_cost<=7000) else 
                                        np.nan), axis=1)
        result_df = result_df.append(data_8)
    if data_9.shape[0] != 0:
        data_9['budget'] = data_9.apply(lambda x: 320000 if (x.create_role_roi>=0.025)&(x.create_role_pay_cost<=5000) else
                                       (320000 if (x.create_role_roi>=0.03)&(x.create_role_pay_cost<=7000) else 
                                        np.nan), axis=1)
        result_df = result_df.append(data_9)
    if data_10.shape[0] != 0:
        data_10['budget'] = data_10.apply(lambda x: 450000 if (x.create_role_roi>=0.026)&(x.create_role_pay_cost<=4000) else
                                       (450000 if (x.create_role_roi>=0.03)&(x.create_role_pay_cost<=6000) else 
                                        np.nan), axis=1)
        result_df = result_df.append(data_10)
    
    # ios加预算
    data_1_ios = result_df_ios[result_df_ios['source_run_date_amount']<=5600]
    data_2_ios = result_df_ios[(result_df_ios['source_run_date_amount']>5600)&(result_df_ios['source_run_date_amount']<=10000)]
    data_3_ios = result_df_ios[(result_df_ios['source_run_date_amount']>10000)&(result_df_ios['source_run_date_amount']<=20000)]
    data_4_ios = result_df_ios[(result_df_ios['source_run_date_amount']>20000)&(result_df_ios['source_run_date_amount']<=46000)]
    data_5_ios = result_df_ios[(result_df_ios['source_run_date_amount']>46000)&(result_df_ios['source_run_date_amount']<=90000)]
    data_6_ios = result_df_ios[(result_df_ios['source_run_date_amount']>90000)&(result_df_ios['source_run_date_amount']<=140000)]
    data_7_ios = result_df_ios[(result_df_ios['source_run_date_amount']>140000)&(result_df_ios['source_run_date_amount']<=200000)]
    data_8_ios = result_df_ios[(result_df_ios['source_run_date_amount']>200000)&(result_df_ios['source_run_date_amount']<=250000)]
    data_9_ios = result_df_ios[(result_df_ios['source_run_date_amount']>250000)&(result_df_ios['source_run_date_amount']<=300000)]
    data_10_ios = result_df_ios[result_df_ios['source_run_date_amount']>300000]
    
    if data_1_ios.shape[0] != 0:
        data_1_ios['budget'] = data_1_ios.apply(lambda x: 8000 if (x.create_role_pay_sum>=60)&(x.create_role_pay_num>=2) else
                                       (8000 if x.create_role_pay_sum>=128 else np.nan), axis=1)
        result_df = result_df.append(data_1_ios)
    if data_2_ios.shape[0] != 0:    
        data_2_ios['budget'] = data_2_ios.apply(lambda x: 13000 if (x.create_role_roi>=0.018)&(x.create_role_pay_num>=2) else
                                        (13000 if x.create_role_roi>=0.3 else np.nan), axis=1)
        result_df = result_df.append(data_2_ios)         
    if data_3_ios.shape[0] != 0:
        data_3_ios['budget'] = data_3_ios.apply(lambda x: 23000 if (x.create_role_roi>=0.019)&(x.create_role_pay_num>=2) else
                                        np.nan, axis=1)
        result_df = result_df.append(data_3_ios)
    if data_4_ios.shape[0] != 0:
        data_4_ios['budget'] = data_4_ios.apply(lambda x: 50000 if (x.create_role_roi>=0.02)&(x.create_role_pay_cost<=9000) else
                                       (50000 if (x.create_role_roi>=0.03)&(x.create_role_pay_cost<=11000) else 
                                        np.nan), axis=1)
        result_df = result_df.append(data_4_ios)
    if data_5_ios.shape[0] != 0:
        data_5_ios['budget'] = data_5_ios.apply(lambda x: 95000 if (x.create_role_roi>=0.02)&(x.create_role_pay_cost<=9000) else
                                       (95000 if (x.create_role_roi>=0.03)&(x.create_role_pay_cost<=11000) else 
                                        np.nan), axis=1)
        result_df = result_df.append(data_5_ios)
    if data_6_ios.shape[0] != 0:
        data_6_ios['budget'] = data_6_ios.apply(lambda x: 148000 if (x.create_role_roi>=0.021)&(x.create_role_pay_cost<=8000) else
                                       (148000 if (x.create_role_roi>=0.03)&(x.create_role_pay_cost<=10000) else 
                                        np.nan), axis=1)
        result_df = result_df.append(data_6_ios)
    if data_7_ios.shape[0] != 0:
        data_7_ios['budget'] = data_7_ios.apply(lambda x: 220000 if (x.create_role_roi>=0.021)&(x.create_role_pay_cost<=8000) else
                                       (220000 if (x.create_role_roi>=0.03)&(x.create_role_pay_cost<=10000) else 
                                        np.nan), axis=1)
        result_df = result_df.append(data_7_ios)
    if data_8_ios.shape[0] != 0:
        data_8_ios['budget'] = data_8_ios.apply(lambda x: 275000 if (x.create_role_roi>=0.022)&(x.create_role_pay_cost<=7000) else
                                       (275000 if (x.create_role_roi>=0.03)&(x.create_role_pay_cost<=9000) else 
                                        np.nan), axis=1)
        result_df = result_df.append(data_8_ios)
    if data_9_ios.shape[0] != 0:
        data_9_ios['budget'] = data_9_ios.apply(lambda x: 320000 if (x.create_role_roi>=0.022)&(x.create_role_pay_cost<=7000) else
                                       (320000 if (x.create_role_roi>=0.03)&(x.create_role_pay_cost<=9000) else 
                                        np.nan), axis=1)
        result_df = result_df.append(data_9_ios)
    if data_10_ios.shape[0] != 0:
        data_10_ios['budget'] = data_10_ios.apply(lambda x: 450000 if (x.create_role_roi>=0.022)&(x.create_role_pay_cost<=6000) else
                                       (450000 if (x.create_role_roi>=0.03)&(x.create_role_pay_cost<=8000) else 
                                        np.nan), axis=1)
        result_df = result_df.append(data_10_ios)
    
    result_df['label'] = result_df['label'].astype(int)
    return {"columns": result_df.columns.values.tolist(), "list": result_df.values.tolist()}

In [227]:
jsondata = {"columns": state_1_data.columns,
           "data":state_1_data.values}
columns_1 = source_predict_state_1(jsondata)['columns']
data_1 = source_predict_state_1(jsondata)['list']
df1 = pd.DataFrame(data_1,columns=columns_1)
df1

,channel_id,source_id,plan_name,model_run_datetime,create_time,media_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_pay_rate,create_role_pay_num_cum,learning_type,learning_time_dt,learning_time_hr,deep_bid_type,label,budget
0,20451,493367,303-幸存者-ROI自动02.18-5,2021-02-22 00:00:00,2021-02-18 00:00:00,10,1001379,2,300001,349,299,3,3200,200,0.026,0,0,2,NaN,NaN,BID_PER_ACTION,0,450000


In [226]:
state_1_data = pd.DataFrame(data=[[
                20451,
                493367,
                "303-幸存者-ROI自动02.18-5",
                "2021-02-22 00:00:00",
                "2021-02-18 00:00:00",
                10,
                1001379,
                2,
                300001,
                349,
                299,
                3,
                3200,
                200,
                0.026,
                0,
                0,
                2,
                np.nan,
                np.nan,
                "BID_PER_ACTION"
            ]],
            columns = [
            "channel_id",
            "source_id",
            "plan_name",
            "model_run_datetime",
            "create_time",
            "media_id",
            "game_id",
            "platform",
            "source_run_date_amount",
            "create_role_num",
            "create_role_cost",
            "create_role_pay_num",
            "create_role_pay_cost",
            "create_role_pay_sum",
            "create_role_roi",
            "create_role_pay_rate",
            "create_role_pay_num_cum",
            "learning_type",
            "learning_time_dt",
            "learning_time_hr",
            "deep_bid_type"])

In [81]:
state_1_data = pd.DataFrame(data=[[
                20451,
                493367,
                "303-幸存者-ROI自动02.18-5",
                "2021-02-22 00:00:00",
                "2021-02-18 00:00:00",
                10,
                1001379,
                1,
                2000,
                349,
                301,
                0,
                6112.46,
                100,
                0,
                0,
                0,
                2,
                np.nan,
                np.nan,
                "BID_PER_ACTION"
            ],
            [
                20451,
                493367,
                "303-幸存者-ROI自动02.18-5",
                "2021-02-22 00:00:00",
                "2021-02-18 00:00:00",
                10,
                1001379,
                1,
                6112.46,
                349,
                101,
                5,
                1222.492,
                180.0,
                0.029448,
                0.01433,
                19,
                2,
                np.nan,
                np.nan,
                "ROI_PACING"
            ],
            [
                20451,
                498996,
                "幸存者-ROI素材_0221_78286430",
                "2021-02-22 00:00:00",
                "2021-02-21 00:00:00",
                10,
                1001379,
                1,
                3000,
                100,
                67.138,
                2,
                6713.8,
                98.0,
                0.0145968,
                0.01,
                3,
                0,
                np.nan,
                np.nan,
                "BID_PER_ACTION"
            ],
            [
                20451,
                499074,
                "306-幸存者-ROI02.21_5",
                "2021-02-22 00:00:00",
                "2021-02-21 00:00:00",
                10,
                1001379,
                1,
                3311.96,
                87,
                38.0685057,
                1,
                3311.96,
                30.0,
                0.0090581,
                0.01149,
                2,
                0,
                np.nan,
                np.nan,
                "ROI_PACING"]],
            columns = [
            "channel_id",
            "source_id",
            "plan_name",
            "model_run_datetime",
            "create_time",
            "media_id",
            "game_id",
            "platform",
            "source_run_date_amount",
            "create_role_num",
            "create_role_cost",
            "create_role_pay_num",
            "create_role_pay_cost",
            "create_role_pay_sum",
            "create_role_roi",
            "create_role_pay_rate",
            "create_role_pay_num_cum",
            "learning_type",
            "learning_time_dt",
            "learning_time_hr",
            "deep_bid_type"])

In [70]:
state_1_data

,channel_id,source_id,plan_name,model_run_datetime,create_time,media_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_pay_rate,create_role_pay_num_cum,learning_type,learning_time_dt,learning_time_hr,deep_bid_type
0,20451,493367,303-幸存者-ROI自动02.18-5,2021-02-22 00:00:00,2021-02-18 00:00:00,10,1001379,1,2000.00,349,301.000000,0,6112.460,100.0,0.000000,0.00000,0,2,NaN,NaN,BID_PER_ACTION
1,20451,493367,303-幸存者-ROI自动02.18-5,2021-02-22 00:00:00,2021-02-18 00:00:00,10,1001379,1,6112.46,349,101.000000,5,1222.492,180.0,0.029448,0.01433,19,2,NaN,NaN,ROI_PACING
2,20451,498996,幸存者-ROI素材_0221_78286430,2021-02-22 00:00:00,2021-02-21 00:00:00,10,1001379,1,6713.80,100,67.138000,1,6713.800,98.0,0.014597,0.01000,3,0,NaN,NaN,BID_PER_ACTION
3,20451,499074,306-幸存者-ROI02.21_5,2021-02-22 00:00:00,2021-02-21 00:00:00,10,1001379,1,3311.96,87,38.068506,1,3311.960,30.0,0.009058,0.01149,2,0,NaN,NaN,ROI_PACING


In [9]:
def source_predict_state_2(jsondata):
    '''
    :param jsondata: {"columns": ["channel_id","source_id","plan_name","model_run_datetime","create_time",
    "media_id","game_id","platform","data_win","source_run_date_amount","create_role_num",
    "create_role_cost","create_role_pay_num","create_role_pay_cost","create_role_pay_sum","create_role_roi",
    "create_role_retain_1d","create_role_pay_rate","create_role_pay_num_cum","learning_type","learning_time_dt",
    "learning_time_hr","deep_bid_type"]}
    :return:
    '''
    data = pd.DataFrame(data=jsondata['data'], columns=jsondata['columns'])

    # 数据预处理
    data = data[data['source_run_date_amount'] > 0]  # 删除消耗为0的数据
    # 去重，处理某天没有消耗的情况
    data.sort_values(by='data_win', inplace=True)
    data.drop_duplicates(subset=['channel_id', 'source_id', 'source_run_date_amount', 'create_role_num'], keep='first'
                         , inplace=True)
    # 对消耗300元以下的不作判断，对数据按平台、win进行分类处理
    data_not_ignore = data[(data['source_run_date_amount'] >= 300) | (data['data_win'] == 1)]
    data_ignore = data.drop(data_not_ignore.index)

    data_win_1 = data_not_ignore[(data_not_ignore['data_win'] == 1) & (data_not_ignore['platform'] == 1)]
    data_win_2 = data_not_ignore[(data_not_ignore['data_win'] == 2) & (data_not_ignore['platform'] == 1)]
    data_win_3 = data_not_ignore[(data_not_ignore['data_win'] == 3) & (data_not_ignore['platform'] == 1)]

    data_win_1_ios = data_not_ignore[(data_not_ignore['data_win'] == 1) & (data_not_ignore['platform'] == 2)]
    data_win_2_ios = data_not_ignore[(data_not_ignore['data_win'] == 2) & (data_not_ignore['platform'] == 2)]
    data_win_3_ios = data_not_ignore[(data_not_ignore['data_win'] == 3) & (data_not_ignore['platform'] == 2)]

    # win=1预判
    result_win1 = pd.DataFrame()
    temp_win1 = pd.DataFrame()

    # data_win_1中再分deep_bid_type
    data_win_1_pre_action = data_win_1[data_win_1['deep_bid_type'] == 'BID_PER_ACTION']
    data_win_1 = data_win_1[data_win_1['deep_bid_type'] != 'BID_PER_ACTION']

    if data_win_1_pre_action.shape[0] != 0:
        data_win_1_pre_action['label'] = data_win_1_pre_action.apply(lambda x: 1 if x.create_role_num == 0 else (1 if
         x.create_role_roi <= 0.02 else (0 if (x.create_role_pay_num > 1) & (
            x.create_role_roi >= 0.022) & (x.source_run_date_amount >= 500) else (
                0 if (x.create_role_roi >= 0.03) & (x.source_run_date_amount >= 500) else 1 if
                   (x.create_role_pay_cost > 8000) | (x.create_role_cost > 220) else 2))), axis=1)

        result_win1 = result_win1.append(
            data_win_1_pre_action[(data_win_1_pre_action['label'] == 1) | (data_win_1_pre_action['label'] == 0)])
        temp_win1 = temp_win1.append(data_win_1_pre_action[data_win_1_pre_action['label'] == 2])

    if data_win_1.shape[0] != 0:
        data_win_1['label'] = data_win_1.apply(lambda x: 1 if x.create_role_num == 0 else (1 if
         x.create_role_roi <= 0.02 else (0 if (x.create_role_pay_num > 1) & (
            x.create_role_roi >= 0.022) & (x.source_run_date_amount >= 500) else (
                0 if (x.create_role_roi >= 0.03) & (x.source_run_date_amount >= 500) else 1 if
                   (x.create_role_pay_cost > 8000) | (x.create_role_cost > 100) else 2))), axis=1)

        result_win1 = result_win1.append(data_win_1[(data_win_1['label'] == 1) | (data_win_1['label'] == 0)])
        temp_win1 = temp_win1.append(data_win_1[data_win_1['label'] == 2])

    data_win_1_ios_pre_action = data_win_1_ios[data_win_1_ios['deep_bid_type'] == 'BID_PER_ACTION']
    data_win_1_ios = data_win_1_ios[data_win_1_ios['deep_bid_type'] != 'BID_PER_ACTION']

    if data_win_1_ios_pre_action.shape[0] != 0:
        data_win_1_ios_pre_action['label'] = data_win_1_ios_pre_action.apply(lambda x: 1 if x.create_role_num == 0 else (1 if
         x.create_role_roi <= 0.018 else (0 if (x.create_role_pay_num > 1) & (
            x.create_role_roi >= 0.02) & (x.source_run_date_amount >= 500) else (
                0 if (x.create_role_roi >= 0.03) & (x.source_run_date_amount >= 500) else 1 if
                   (x.create_role_pay_cost > 12000) | (x.create_role_cost > 300) else 2))), axis=1)

        result_win1 = result_win1.append(
            data_win_1_ios_pre_action[(data_win_1_ios_pre_action['label'] == 1) | (data_win_1_ios_pre_action['label'] == 0)])
        temp_win1 = temp_win1.append(data_win_1_ios_pre_action[data_win_1_ios_pre_action['label'] == 2])

    if data_win_1_ios.shape[0] != 0:
        data_win_1_ios['label'] = data_win_1_ios.apply(lambda x: 1 if x.create_role_num == 0 else (1 if
         x.create_role_roi <= 0.018 else (0 if (x.create_role_pay_num > 1) & (
            x.create_role_roi >= 0.02) & (x.source_run_date_amount >= 500) else (
                0 if (x.create_role_roi >= 0.03) & (x.source_run_date_amount) >= 500 else 1 if
                   (x.create_role_pay_cost > 12000) | (x.create_role_cost > 200) else 2))), axis=1)

        result_win1 = result_win1.append(
            data_win_1_ios[(data_win_1_ios['label'] == 1) | (data_win_1_ios['label'] == 0)])
        temp_win1 = temp_win1.append(data_win_1_ios[data_win_1_ios['label'] == 2])

    # win=1 模型预测
    if temp_win1.shape[0] != 0:
        temp_win1['label'] = gbdt_b_win1.predict(temp_win1[['create_role_cost',
                                                            'create_role_pay_cost', 'create_role_roi',
                                                            'create_role_pay_rate']])
    result_win1_data = pd.concat([result_win1, temp_win1], axis=0)

    # win=2预判
    result_win2 = pd.DataFrame()
    temp_win2 = pd.DataFrame()
    if data_win_2.shape[0] != 0:
        data_win_2['label'] = data_win_2.apply(lambda x: 1 if x.create_role_num == 0 else (1 if
         x.create_role_roi <= 0.05 else (0 if (x.create_role_pay_num > 1) & (
            x.create_role_roi >= 0.06) & (x.source_run_date_amount >= 500) else (
                0 if (x.create_role_roi >= 0.07) & (x.source_run_date_amount >= 500) else 1 if
                  (x.create_role_pay_cost > 7000) | (x.create_role_cost > 150) else 2))), axis=1)

        result_win2 = result_win2.append(data_win_2[(data_win_2['label'] == 1) | (data_win_2['label'] == 0)])
        temp_win2 = temp_win2.append(data_win_2[data_win_2['label'] == 2])

    if data_win_2_ios.shape[0] != 0:
        data_win_2_ios['label'] = data_win_2_ios.apply(lambda x: 1 if x.create_role_num == 0 else (1 if
           x.create_role_roi <= 0.045 else (0 if (x.create_role_pay_num > 1) & (
             x.create_role_roi >= 0.05) & (x.source_run_date_amount >= 500) else (
                0 if (x.create_role_roi >= 0.065) & (x.source_run_date_amount >= 500) else 1 if
                (x.create_role_pay_cost > 10000) | (x.create_role_cost > 250) else 2))), axis=1)

        result_win2 = result_win2.append(data_win_2_ios[(data_win_2_ios['label'] == 1) | (data_win_2_ios['label'] == 0)])
        temp_win2 = temp_win2.append(data_win_2_ios[data_win_2_ios['label'] == 2])

    # win=2 模型预测
    if temp_win2.shape[0] != 0:
        temp_win2['label'] = gbdt_b_win2.predict(temp_win2[['create_role_cost',
                                                            'create_role_pay_cost', 'create_role_roi',
                                                            'create_role_pay_rate', 'create_role_retain_1d']])
    result_win2_data = pd.concat([result_win2, temp_win2], axis=0)

    # win=3预判
    result_win3 = pd.DataFrame()
    temp_win3 = pd.DataFrame()
    if data_win_3.shape[0] != 0:
        data_win_3['label'] = data_win_3.apply(lambda x: 1 if x.create_role_num == 0 else (1 if
           x.create_role_roi <= 0.07 else (
            0 if (x.create_role_pay_num > 1) & (
                    x.create_role_roi >= 0.09) & (x.source_run_date_amount >= 500) else (
                0 if (x.create_role_roi >= 0.11) & (x.source_run_date_amount >= 500) else 1 if
                (x.create_role_pay_cost > 6000) | (x.create_role_cost > 120) else 2))), axis=1)

        result_win3 = result_win3.append(data_win_3[(data_win_3['label'] == 1) | (data_win_3['label'] == 0)])
        temp_win3 = temp_win3.append(data_win_3[data_win_3['label'] == 2])

    if data_win_3_ios.shape[0] != 0:
        data_win_3_ios['label'] = data_win_3_ios.apply(lambda x: 1 if x.create_role_num == 0 else (1 if
           x.create_role_roi <= 0.06 else (0 if (x.create_role_pay_num > 1) & (
              x.create_role_roi >= 0.07) & (x.source_run_date_amount >= 500) else (
                0 if (x.create_role_roi >= 0.08) & (x.source_run_date_amount >= 500) else 1 if
                (x.create_role_pay_cost > 9000) | (x.create_role_cost > 200) else 2))), axis=1)

        result_win3 = result_win3.append(
            data_win_3_ios[(data_win_3_ios['label'] == 1) | (data_win_3_ios['label'] == 0)])
        temp_win3 = temp_win3.append(data_win_3_ios[data_win_3_ios['label'] == 2])

    # win=3 模型预测
    if temp_win3.shape[0] != 0:
        temp_win3['label'] = gbdt_b_win3.predict(temp_win3[['create_role_cost',
                                                            'create_role_pay_cost', 'create_role_roi',
                                                            'create_role_pay_rate', 'create_role_retain_1d']])
    result_win3_data = pd.concat([result_win3, temp_win3], axis=0)

    result = pd.concat([result_win1_data, result_win2_data, result_win3_data], axis=0)

    # 结果输出
    if result.shape[0] != 0:
        result_1_2_3 = result[result['data_win'] != 7]
        # win1,2,3按优先级3,2,1进行去重
        if result_1_2_3.shape[0] != 0:
            result_1_2_3.sort_values('data_win', ascending=False, inplace=True)
            result_1_2_3.drop_duplicates(['channel_id', 'source_id', 'model_run_datetime'], keep='first', inplace=True)

        # # 再与win=7进行去重，win=7优先级小
        # result_all = pd.concat([result_1_2_3, result[result['data_win'] == 7]], axis=0)
        # 
        # result_all.sort_values('data_win', inplace=True)
        source_predict = result_1_2_3.drop_duplicates(['channel_id', 'source_id', 'model_run_datetime'], keep='first')

    else:
        source_predict = pd.DataFrame(
            columns=["channel_id", "source_id", "plan_name", "model_run_datetime", "create_time",
                     "media_id", "game_id", "platform", "data_win", "source_run_date_amount", "create_role_num",
                     "create_role_cost", "create_role_pay_num", "create_role_pay_cost", "create_role_pay_sum",
                     "create_role_roi",
                     "create_role_retain_1d", "create_role_pay_rate", "create_role_pay_num_cum", "learning_type",
                     "learning_time_dt",
                     "learning_time_hr", "deep_bid_type", "learning_time", "learning_time_interval", "label"])

    if data_ignore.shape[0] != 0:
        data_ignore['label'] = 1
        data_ignore.sort_values('data_win', ascending=False, inplace=True)
        data_ignore = data_ignore.drop_duplicates(['channel_id', 'source_id', 'model_run_datetime'], keep='first')
        data_ignore['data_win'] = -1

    source_predict = pd.concat([source_predict, data_ignore], axis=0)
    source_predict.sort_values('data_win', ascending=False, inplace=True)
    source_predict = source_predict.drop_duplicates(['channel_id', 'source_id', 'model_run_datetime'], keep='first')

    source_predict['label'] = source_predict['label'].astype(int)

    return {"columns": source_predict.columns.values.tolist(), "list": source_predict.values.tolist()}

In [88]:
jsondata = {"columns": state_2_data.columns,
           "data":state_2_data.values}
columns_2 = source_predict_state_2(jsondata)['columns']
data_2 = source_predict_state_2(jsondata)['list']
df2 = pd.DataFrame(data_2,columns=columns_2)
df2

TypeError: unsupported operand type(s) for &: 'bool' and 'float'

In [89]:
state_2_data

,channel_id,source_id,plan_name,model_run_datetime,create_time,media_id,game_id,platform,data_win,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d,create_role_pay_rate,create_role_pay_num_cum,learning_type,learning_time_dt,learning_time_hr,deep_bid_type
0,20541,506968,RICH-财神到 -研究-通投_横SSR2138_0226_16,2021-03-16 00:00:00,2021-02-26 00:00:00,10,1001441,2,1,8411.22,103,81.66233,1,8411.22,192.0,0.022827,0.0,0.00971,3,2,NaN,NaN,DEEP_BID_DEFAULT


In [87]:
state_2_data = pd.DataFrame(data=[
           [20541,506968,"RICH-财神到 -研究-通投_横SSR2138_0226_16","2021-03-16 00:00:00","2021-02-26 00:00:00",10,1001441,2,1,8411.22,103,81.6623301,1,8411.220000,192.00,0.0228267,0.0000,0.00971,3,2,np.nan,np.nan,"DEEP_BID_DEFAULT"]],
            columns = [
            "channel_id",
            "source_id",
            "plan_name",
            "model_run_datetime",
            "create_time",
            "media_id",
            "game_id",
            "platform",
            "data_win",
            "source_run_date_amount",
            "create_role_num",
            "create_role_cost",
            "create_role_pay_num",
            "create_role_pay_cost",
            "create_role_pay_sum",
            "create_role_roi",
            "create_role_retain_1d",
            "create_role_pay_rate",
            "create_role_pay_num_cum",
            "learning_type",
            "learning_time_dt",
            "learning_time_hr",
            "deep_bid_type"])

In [23]:
s = pd.DataFrame({'A':[1,2,np.nan]})

In [24]:
s

,A
0,1.0
1,2.0
2,NaN


In [25]:
s['A'] = s['A'].replace(np.nan, 'None')

In [26]:
s

,A
0,1
1,2
2,None


In [27]:
import tensorflow as tf

In [30]:
feature_embedding = tf.constant(np.random.randint(10,size=(10,5)))

In [31]:
feature_embedding

<tf.Tensor: shape=(10, 5), dtype=int32, numpy=
array([[4, 5, 7, 9, 3],
       [4, 9, 5, 3, 8],
       [5, 7, 6, 4, 6],
       [9, 5, 1, 1, 5],
       [5, 9, 4, 8, 0],
       [4, 6, 7, 1, 1],
       [4, 0, 2, 8, 9],
       [6, 1, 5, 1, 5],
       [1, 8, 7, 5, 7],
       [4, 3, 1, 0, 7]])>

In [32]:
feat_index = tf.constant(np.random.randint(10,size=(20,6)))

In [33]:
feat_index

<tf.Tensor: shape=(20, 6), dtype=int32, numpy=
array([[8, 0, 7, 2, 3, 5],
       [5, 5, 0, 8, 7, 7],
       [9, 9, 1, 5, 1, 4],
       [2, 6, 5, 7, 2, 2],
       [1, 6, 4, 3, 6, 6],
       [6, 9, 6, 0, 1, 9],
       [3, 7, 6, 9, 3, 6],
       [2, 2, 1, 8, 5, 9],
       [0, 2, 6, 1, 5, 2],
       [1, 3, 3, 9, 3, 2],
       [8, 0, 3, 5, 9, 8],
       [4, 3, 3, 8, 6, 8],
       [6, 9, 8, 3, 6, 0],
       [3, 9, 5, 4, 6, 3],
       [4, 7, 9, 9, 8, 5],
       [8, 5, 1, 4, 8, 9],
       [4, 2, 6, 8, 9, 1],
       [0, 6, 4, 9, 7, 1],
       [4, 3, 3, 5, 8, 3],
       [3, 3, 7, 6, 8, 7]])>

In [39]:
embedding=tf.nn.embedding_lookup(feature_embedding,feat_index)

In [40]:
embedding

<tf.Tensor: shape=(20, 6, 5), dtype=int32, numpy=
array([[[1, 8, 7, 5, 7],
        [4, 5, 7, 9, 3],
        [6, 1, 5, 1, 5],
        [5, 7, 6, 4, 6],
        [9, 5, 1, 1, 5],
        [4, 6, 7, 1, 1]],

       [[4, 6, 7, 1, 1],
        [4, 6, 7, 1, 1],
        [4, 5, 7, 9, 3],
        [1, 8, 7, 5, 7],
        [6, 1, 5, 1, 5],
        [6, 1, 5, 1, 5]],

       [[4, 3, 1, 0, 7],
        [4, 3, 1, 0, 7],
        [4, 9, 5, 3, 8],
        [4, 6, 7, 1, 1],
        [4, 9, 5, 3, 8],
        [5, 9, 4, 8, 0]],

       [[5, 7, 6, 4, 6],
        [4, 0, 2, 8, 9],
        [4, 6, 7, 1, 1],
        [6, 1, 5, 1, 5],
        [5, 7, 6, 4, 6],
        [5, 7, 6, 4, 6]],

       [[4, 9, 5, 3, 8],
        [4, 0, 2, 8, 9],
        [5, 9, 4, 8, 0],
        [9, 5, 1, 1, 5],
        [4, 0, 2, 8, 9],
        [4, 0, 2, 8, 9]],

       [[4, 0, 2, 8, 9],
        [4, 3, 1, 0, 7],
        [4, 0, 2, 8, 9],
        [4, 5, 7, 9, 3],
        [4, 9, 5, 3, 8],
        [4, 3, 1, 0, 7]],

       [[9, 5, 1, 1, 5],
        [6, 1

In [46]:
feat_value = tf.ones_like(np.random.randint(10,size=(20,6)))

In [54]:
arr = np.ones_like(np.random.randint(10,size=(20,6)))
arr[:,3] = np.arange(20)

In [56]:
feat_value = tf.constant(arr)

In [57]:
feat_value

<tf.Tensor: shape=(20, 6), dtype=int32, numpy=
array([[ 1,  1,  1,  0,  1,  1],
       [ 1,  1,  1,  1,  1,  1],
       [ 1,  1,  1,  2,  1,  1],
       [ 1,  1,  1,  3,  1,  1],
       [ 1,  1,  1,  4,  1,  1],
       [ 1,  1,  1,  5,  1,  1],
       [ 1,  1,  1,  6,  1,  1],
       [ 1,  1,  1,  7,  1,  1],
       [ 1,  1,  1,  8,  1,  1],
       [ 1,  1,  1,  9,  1,  1],
       [ 1,  1,  1, 10,  1,  1],
       [ 1,  1,  1, 11,  1,  1],
       [ 1,  1,  1, 12,  1,  1],
       [ 1,  1,  1, 13,  1,  1],
       [ 1,  1,  1, 14,  1,  1],
       [ 1,  1,  1, 15,  1,  1],
       [ 1,  1,  1, 16,  1,  1],
       [ 1,  1,  1, 17,  1,  1],
       [ 1,  1,  1, 18,  1,  1],
       [ 1,  1,  1, 19,  1,  1]])>

In [58]:
feat_value = tf.reshape(feat_value,shape=[-1,6,1])

In [59]:
embeddings = tf.multiply(embedding,feat_value)

In [60]:
embedding

<tf.Tensor: shape=(20, 6, 5), dtype=int32, numpy=
array([[[1, 8, 7, 5, 7],
        [4, 5, 7, 9, 3],
        [6, 1, 5, 1, 5],
        [5, 7, 6, 4, 6],
        [9, 5, 1, 1, 5],
        [4, 6, 7, 1, 1]],

       [[4, 6, 7, 1, 1],
        [4, 6, 7, 1, 1],
        [4, 5, 7, 9, 3],
        [1, 8, 7, 5, 7],
        [6, 1, 5, 1, 5],
        [6, 1, 5, 1, 5]],

       [[4, 3, 1, 0, 7],
        [4, 3, 1, 0, 7],
        [4, 9, 5, 3, 8],
        [4, 6, 7, 1, 1],
        [4, 9, 5, 3, 8],
        [5, 9, 4, 8, 0]],

       [[5, 7, 6, 4, 6],
        [4, 0, 2, 8, 9],
        [4, 6, 7, 1, 1],
        [6, 1, 5, 1, 5],
        [5, 7, 6, 4, 6],
        [5, 7, 6, 4, 6]],

       [[4, 9, 5, 3, 8],
        [4, 0, 2, 8, 9],
        [5, 9, 4, 8, 0],
        [9, 5, 1, 1, 5],
        [4, 0, 2, 8, 9],
        [4, 0, 2, 8, 9]],

       [[4, 0, 2, 8, 9],
        [4, 3, 1, 0, 7],
        [4, 0, 2, 8, 9],
        [4, 5, 7, 9, 3],
        [4, 9, 5, 3, 8],
        [4, 3, 1, 0, 7]],

       [[9, 5, 1, 1, 5],
        [6, 1

In [61]:
feat_value

<tf.Tensor: shape=(20, 6, 1), dtype=int32, numpy=
array([[[ 1],
        [ 1],
        [ 1],
        [ 0],
        [ 1],
        [ 1]],

       [[ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1]],

       [[ 1],
        [ 1],
        [ 1],
        [ 2],
        [ 1],
        [ 1]],

       [[ 1],
        [ 1],
        [ 1],
        [ 3],
        [ 1],
        [ 1]],

       [[ 1],
        [ 1],
        [ 1],
        [ 4],
        [ 1],
        [ 1]],

       [[ 1],
        [ 1],
        [ 1],
        [ 5],
        [ 1],
        [ 1]],

       [[ 1],
        [ 1],
        [ 1],
        [ 6],
        [ 1],
        [ 1]],

       [[ 1],
        [ 1],
        [ 1],
        [ 7],
        [ 1],
        [ 1]],

       [[ 1],
        [ 1],
        [ 1],
        [ 8],
        [ 1],
        [ 1]],

       [[ 1],
        [ 1],
        [ 1],
        [ 9],
        [ 1],
        [ 1]],

       [[ 1],
        [ 1],
        [ 1],
        [10],
        [ 1],
        [ 1]],

    

In [62]:
embeddings

<tf.Tensor: shape=(20, 6, 5), dtype=int32, numpy=
array([[[  1,   8,   7,   5,   7],
        [  4,   5,   7,   9,   3],
        [  6,   1,   5,   1,   5],
        [  0,   0,   0,   0,   0],
        [  9,   5,   1,   1,   5],
        [  4,   6,   7,   1,   1]],

       [[  4,   6,   7,   1,   1],
        [  4,   6,   7,   1,   1],
        [  4,   5,   7,   9,   3],
        [  1,   8,   7,   5,   7],
        [  6,   1,   5,   1,   5],
        [  6,   1,   5,   1,   5]],

       [[  4,   3,   1,   0,   7],
        [  4,   3,   1,   0,   7],
        [  4,   9,   5,   3,   8],
        [  8,  12,  14,   2,   2],
        [  4,   9,   5,   3,   8],
        [  5,   9,   4,   8,   0]],

       [[  5,   7,   6,   4,   6],
        [  4,   0,   2,   8,   9],
        [  4,   6,   7,   1,   1],
        [ 18,   3,  15,   3,  15],
        [  5,   7,   6,   4,   6],
        [  5,   7,   6,   4,   6]],

       [[  4,   9,   5,   3,   8],
        [  4,   0,   2,   8,   9],
        [  5,   9,   4,   8,   0

In [63]:
feature_bias = tf.constant(np.random.randint(10,size=(10,1)))

In [64]:
feature_bias

<tf.Tensor: shape=(10, 1), dtype=int32, numpy=
array([[5],
       [8],
       [6],
       [9],
       [0],
       [6],
       [3],
       [8],
       [3],
       [4]])>

In [65]:
y_first_order = tf.nn.embedding_lookup(feature_bias,feat_index)

In [66]:
y_first_order

<tf.Tensor: shape=(20, 6, 1), dtype=int32, numpy=
array([[[3],
        [5],
        [8],
        [6],
        [9],
        [6]],

       [[6],
        [6],
        [5],
        [3],
        [8],
        [8]],

       [[4],
        [4],
        [8],
        [6],
        [8],
        [0]],

       [[6],
        [3],
        [6],
        [8],
        [6],
        [6]],

       [[8],
        [3],
        [0],
        [9],
        [3],
        [3]],

       [[3],
        [4],
        [3],
        [5],
        [8],
        [4]],

       [[9],
        [8],
        [3],
        [4],
        [9],
        [3]],

       [[6],
        [6],
        [8],
        [3],
        [6],
        [4]],

       [[5],
        [6],
        [3],
        [8],
        [6],
        [6]],

       [[8],
        [9],
        [9],
        [4],
        [9],
        [6]],

       [[3],
        [5],
        [9],
        [6],
        [4],
        [3]],

       [[0],
        [9],
        [9],
        [3],
        [3],
     

In [67]:
feat_value

<tf.Tensor: shape=(20, 6, 1), dtype=int32, numpy=
array([[[ 1],
        [ 1],
        [ 1],
        [ 0],
        [ 1],
        [ 1]],

       [[ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1]],

       [[ 1],
        [ 1],
        [ 1],
        [ 2],
        [ 1],
        [ 1]],

       [[ 1],
        [ 1],
        [ 1],
        [ 3],
        [ 1],
        [ 1]],

       [[ 1],
        [ 1],
        [ 1],
        [ 4],
        [ 1],
        [ 1]],

       [[ 1],
        [ 1],
        [ 1],
        [ 5],
        [ 1],
        [ 1]],

       [[ 1],
        [ 1],
        [ 1],
        [ 6],
        [ 1],
        [ 1]],

       [[ 1],
        [ 1],
        [ 1],
        [ 7],
        [ 1],
        [ 1]],

       [[ 1],
        [ 1],
        [ 1],
        [ 8],
        [ 1],
        [ 1]],

       [[ 1],
        [ 1],
        [ 1],
        [ 9],
        [ 1],
        [ 1]],

       [[ 1],
        [ 1],
        [ 1],
        [10],
        [ 1],
        [ 1]],

    

In [70]:
tf.multiply(y_first_order,feat_value)

<tf.Tensor: shape=(20, 6, 1), dtype=int32, numpy=
array([[[  3],
        [  5],
        [  8],
        [  0],
        [  9],
        [  6]],

       [[  6],
        [  6],
        [  5],
        [  3],
        [  8],
        [  8]],

       [[  4],
        [  4],
        [  8],
        [ 12],
        [  8],
        [  0]],

       [[  6],
        [  3],
        [  6],
        [ 24],
        [  6],
        [  6]],

       [[  8],
        [  3],
        [  0],
        [ 36],
        [  3],
        [  3]],

       [[  3],
        [  4],
        [  3],
        [ 25],
        [  8],
        [  4]],

       [[  9],
        [  8],
        [  3],
        [ 24],
        [  9],
        [  3]],

       [[  6],
        [  6],
        [  8],
        [ 21],
        [  6],
        [  4]],

       [[  5],
        [  6],
        [  3],
        [ 64],
        [  6],
        [  6]],

       [[  8],
        [  9],
        [  9],
        [ 36],
        [  9],
        [  6]],

       [[  3],
        [  5],


In [68]:
y_first_orders = tf.reduce_sum(tf.multiply(y_first_order,feat_value),2)

In [69]:
y_first_orders

<tf.Tensor: shape=(20, 6), dtype=int32, numpy=
array([[  3,   5,   8,   0,   9,   6],
       [  6,   6,   5,   3,   8,   8],
       [  4,   4,   8,  12,   8,   0],
       [  6,   3,   6,  24,   6,   6],
       [  8,   3,   0,  36,   3,   3],
       [  3,   4,   3,  25,   8,   4],
       [  9,   8,   3,  24,   9,   3],
       [  6,   6,   8,  21,   6,   4],
       [  5,   6,   3,  64,   6,   6],
       [  8,   9,   9,  36,   9,   6],
       [  3,   5,   9,  60,   4,   3],
       [  0,   9,   9,  33,   3,   3],
       [  3,   4,   3, 108,   3,   5],
       [  9,   4,   6,   0,   3,   9],
       [  0,   8,   4,  56,   3,   6],
       [  3,   6,   8,   0,   3,   4],
       [  0,   6,   3,  48,   4,   8],
       [  5,   3,   0,  68,   8,   8],
       [  0,   9,   9, 108,   3,   9],
       [  9,   9,   8,  57,   3,   8]])>

In [74]:
y_first_order_2 = tf.nn.dropout(y_first_orders,0.1)

ValueError: x has to be a floating point tensor since it's going to be scaled. Got a <dtype: 'int32'> tensor instead.

In [76]:
embeddings

<tf.Tensor: shape=(20, 6, 5), dtype=int32, numpy=
array([[[  1,   8,   7,   5,   7],
        [  4,   5,   7,   9,   3],
        [  6,   1,   5,   1,   5],
        [  0,   0,   0,   0,   0],
        [  9,   5,   1,   1,   5],
        [  4,   6,   7,   1,   1]],

       [[  4,   6,   7,   1,   1],
        [  4,   6,   7,   1,   1],
        [  4,   5,   7,   9,   3],
        [  1,   8,   7,   5,   7],
        [  6,   1,   5,   1,   5],
        [  6,   1,   5,   1,   5]],

       [[  4,   3,   1,   0,   7],
        [  4,   3,   1,   0,   7],
        [  4,   9,   5,   3,   8],
        [  8,  12,  14,   2,   2],
        [  4,   9,   5,   3,   8],
        [  5,   9,   4,   8,   0]],

       [[  5,   7,   6,   4,   6],
        [  4,   0,   2,   8,   9],
        [  4,   6,   7,   1,   1],
        [ 18,   3,  15,   3,  15],
        [  5,   7,   6,   4,   6],
        [  5,   7,   6,   4,   6]],

       [[  4,   9,   5,   3,   8],
        [  4,   0,   2,   8,   9],
        [  5,   9,   4,   8,   0

In [75]:
tf.reduce_sum(embeddings, 1)

<tf.Tensor: shape=(20, 5), dtype=int32, numpy=
array([[ 24,  25,  27,  17,  21],
       [ 25,  27,  38,  18,  22],
       [ 29,  45,  30,  16,  32],
       [ 41,  30,  42,  24,  43],
       [ 57,  38,  19,  39,  55],
       [ 40,  40,  46,  64,  55],
       [ 56,  29,  17,  19,  75],
       [ 29,  88,  74,  47,  77],
       [ 54,  97,  68,  50,  89],
       [ 72,  58,  23,  10,  92],
       [ 59,  89,  93,  30,  39],
       [ 39, 115,  92,  78, 103],
       [125,  76,  31,  42,  95],
       [ 95, 136,  64, 115,  27],
       [ 76,  69,  38,  15, 118],
       [ 89, 169,  87, 134,  30],
       [ 38, 156, 130, 103, 142],
       [ 91,  75,  40,  29, 144],
       [105, 140, 140,  34,  40],
       [107,  20,  57, 161, 198]])>

In [77]:
tf.square(embeddings)

<tf.Tensor: shape=(20, 6, 5), dtype=int32, numpy=
array([[[    1,    64,    49,    25,    49],
        [   16,    25,    49,    81,     9],
        [   36,     1,    25,     1,    25],
        [    0,     0,     0,     0,     0],
        [   81,    25,     1,     1,    25],
        [   16,    36,    49,     1,     1]],

       [[   16,    36,    49,     1,     1],
        [   16,    36,    49,     1,     1],
        [   16,    25,    49,    81,     9],
        [    1,    64,    49,    25,    49],
        [   36,     1,    25,     1,    25],
        [   36,     1,    25,     1,    25]],

       [[   16,     9,     1,     0,    49],
        [   16,     9,     1,     0,    49],
        [   16,    81,    25,     9,    64],
        [   64,   144,   196,     4,     4],
        [   16,    81,    25,     9,    64],
        [   25,    81,    16,    64,     0]],

       [[   25,    49,    36,    16,    36],
        [   16,     0,     4,    64,    81],
        [   16,    36,    49,     1,     1],

In [78]:
tf.reduce_sum(tf.square(embeddings), 1)

<tf.Tensor: shape=(20, 5), dtype=int32, numpy=
array([[  150,   151,   173,   109,   109],
       [  121,   163,   246,   110,   110],
       [  153,   405,   264,    86,   230],
       [  431,   192,   386,   122,   415],
       [ 1385,   562,    69,   281,   707],
       [  480,   724,  1260,  2162,   549],
       [  806,   375,    71,   131,  2001],
       [  147,  3360,  2548,  1267,  2587],
       [ 1122,  5343,  1774,   754,  4259],
       [ 1580,   934,   145,    28,  4144],
       [ 1715,  3787,  5049,   232,   281],
       [  325,  7939,  6000,  3180,  6109],
       [11729,  3698,   251,   378,  3869],
       [ 4435, 13784,  2760, 10883,   181],
       [ 3230,  1955,   336,    91,  9728],
       [ 5675, 18479,  3773, 14460,   212],
       [  354, 16604, 12626,  6553, 12774],
       [ 4733,  2789,   408,   219, 14340],
       [ 5453, 11884, 15944,   416,   448],
       [ 6011,   116,  1545, 23133, 29390]])>

In [93]:
info = pd.DataFrame({'Name': {0: 'Parker', 1: 'Smith', 2: 'John'}, 'Language': {0: 'Python', 1: 'Java', 2: 'C++'}, 'Age': {0: 22, 1: 30, 2: 26}}) 


In [94]:
info

,Name,Language,Age
0,Parker,Python,22
1,Smith,Java,30
2,John,C++,26


In [95]:
pd.melt(info, id_vars =['Name'], value_vars =['Language']) 

,Name,variable,value
0,Parker,Language,Python
1,Smith,Language,Java
2,John,Language,C++


In [97]:
info = pd.DataFrame({'A': {0: 'p', 1: 'q', 2: 'r'}, 'B': {0: 40, 1: 55, 2: 25}, 'C': {0: 56, 1: 62, 2: 42}})

In [98]:
info

,A,B,C
0,p,40,56
1,q,55,62
2,r,25,42


In [99]:
pd.melt(info, id_vars=['A'], value_vars=['C'])

,A,variable,value
0,p,C,56
1,q,C,62
2,r,C,42


In [100]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})


In [101]:
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [102]:
pd.melt(df, id_vars=['key'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [118]:
df.keys

<bound method NDFrame.keys of    key  A  B  C
0  foo  1  4  7
1  bar  2  5  8
2  baz  3  6  9>